In [11]:
import os

# Set the environment variables for XRT
xrt_213_path = "/home/csl/yeop/XRT/build/Debug/opt/xilinx/xrt"

os.environ["XILINX_XRT"] = xrt_213_path
os.environ["LD_LIBRARY_PATH"] = f"{xrt_213_path}/lib:" + os.getenv("LD_LIBRARY_PATH", "")
os.environ["PATH"] = f"{xrt_213_path}/bin:" + os.getenv("PATH", "")

In [12]:
!xbutil --version

Version              : 2.13.0
Branch               : 2022.1
Hash                 : ca4bdf101cd1d698377f5a7113d8583f709c2143
Hash Date            : 2025-02-27 06:03:19
XOCL                 : 2.18.179, 3ade2e671e5ab463400813fc2846c57edf82bb10
XCLMGMT              : 2.18.179, 3ade2e671e5ab463400813fc2846c57edf82bb10


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [14]:
from qkeras import QDense, QActivation, quantized_bits, quantized_relu
from tensorflow.keras.models import Sequential
import numpy as np

X = np.random.rand(1000, 10)
y = np.random.randint(2, size=(1000, 1))

model = Sequential()
model.add(
    QDense(
        64,
        input_shape=(10,),
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(QActivation(activation=quantized_relu(6)))
model.add(
    QDense(
        32,
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(QActivation(activation=quantized_relu(6)))
model.add(
    QDense(
        1,
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform'
    )
)
model.add(Activation('sigmoid'))  # Sigmoid can't be quantized directly (but hls4ml handles it fine)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=5, batch_size=32, verbose=1)

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 0.6948 - accuracy: 0.5160
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.6922 - accuracy: 0.5060
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.6908 - accuracy: 0.5290
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.6891 - accuracy: 0.5320
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.6883 - accuracy: 0.5330


In [15]:
import hls4ml
import pprint

# Generate config
config = hls4ml.utils.config_from_keras_model(model, default_precision='ap_fixed<16,6>')

# Add U55C-specific settings
config['Backend'] = 'Vitis'
config['Part'] = 'xcu55c-fsvh2892-2L-e'
config['ProjectName'] = 'hls4ml_prj_u55c_new'
config['ClockPeriod'] = 5  # You can adjust this if needed
config['IOType'] = 'io_parallel'

pprint.pprint(config)

Interpreting Sequential
Topology:
Layer name: q_dense_3_input, layer type: InputLayer, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 64]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: q_activation_3, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: q_dense_5, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 1]
Layer name: activation_1, layer type: Activation, input shapes: [[None, 1]], output shape: [None, 1]
{'Backend': 'Vitis',
 'ClockPeriod': 5,
 'IOType': 'io_parallel',
 'Model': {'BramFactor': 1000000000,
           'Precision': {'default': 'ap_fixed<16,6>'},
           'ReuseFactor': 1,
           'Strategy': 'Latency',
           'TraceOutput': False},
 'Part'

In [16]:
# Convert the model
hls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config=config,
                                                       io_type='io_parallel',
                                                       output_dir='hls4ml_prj_u55c_new',
                                                       part='xcu55c-fsvh2892-2L-e',
                                                       backend='Vitis',)


Interpreting Sequential
Topology:
Layer name: q_dense_3_input, layer type: InputLayer, input shapes: [[None, 10]], output shape: [None, 10]
Layer name: q_dense_3, layer type: QDense, input shapes: [[None, 10]], output shape: [None, 64]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: q_dense_4, layer type: QDense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: q_activation_3, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: q_dense_5, layer type: QDense, input shapes: [[None, 32]], output shape: [None, 1]
Layer name: activation_1, layer type: Activation, input shapes: [[None, 1]], output shape: [None, 1]
Creating HLS model


In [17]:
hls_model.build(csim=False,synth=True,export=True)

Writing HLS project
Done

****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2022.1 (64-bit)
  **** SW Build 3526262 on Mon Apr 18 15:47:01 MDT 2022
  **** IP Build 3524634 on Mon Apr 18 20:55:01 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source /home/Xilinx/Vitis_HLS/2022.1/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/home/Xilinx/Vitis_HLS/2022.1/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'csl' on host 'gpu2' (Linux_x86_64 version 5.4.0-212-generic) on Wed Apr 16 07:15:27 UTC 2025
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/csl/sky/hls4ml_practice/hls4ml_prj_u55c_new'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project myproject_prj 
INFO: [HLS 200-10] Creating and opening project '/home/csl/sky/hls4ml_practice/hls4ml_prj_u55c_new/myproject_prj'.
INFO: [HLS 200-1510] Running: set_top myproject 
INFO: [HLS 200-1510] Running: add_files firmware/

{'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '3.645',
  'BestLatency': '9',
  'WorstLatency': '9',
  'IntervalMin': '1',
  'IntervalMax': '1',
  'BRAM_18K': '1',
  'DSP': '38',
  'FF': '10525',
  'LUT': '69647',
  'URAM': '0',
  'AvailableBRAM_18K': '4032',
  'AvailableDSP': '9024',
  'AvailableFF': '2607360',
  'AvailableLUT': '1303680',
  'AvailableURAM': '960'}}

In [18]:
!v++ -c -t hw --platform xilinx_u55c_gen3x16_xdma_3_202210_1 -k myproject -o myproject.xo ./hls4ml_prj_u55c_new/firmware/myproject.cpp

Option Map File Used: '/home/Xilinx/Vitis/2022.1/data/vitis/vpp/optMap.xml'

****** v++ v2022.1 (64-bit)
  **** SW Build 3524075 on 2022-04-13-17:42:45
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ compile can be found at:
	Reports: /home/csl/sky/hls4ml_practice/_x/reports/myproject
	Log files: /home/csl/sky/hls4ml_practice/_x/logs/myproject
Running Dispatch Server on port: 37879
INFO: [v++ 60-1548] Creating build summary session with primary output /home/csl/sky/hls4ml_practice/myproject.xo.compile_summary, at Wed Apr 16 07:19:32 2025
INFO: [v++ 60-1316] Initiating connection to rulecheck server, at Wed Apr 16 07:19:32 2025
INFO: [v++ 60-1315] Creating rulecheck session with output '/home/csl/sky/hls4ml_practice/_x/reports/myproject/v++_compile_myproject_guidance.html', at Wed Apr 16 07:19:33 2025
INFO: [v++ 60-895]   Target platform: /opt/xilinx/platforms/xilinx_u55c_gen3x16_xdma_3_202210_1/xilinx_u55

In [20]:
hls4ml.backends.VitisBackend.make_xclbin(hls_model, hls_model, 'xilinx_u55c_gen3x16_xdma_3_202210_1')


****** Vivado v2022.1 (64-bit)
  **** SW Build 3526262 on Mon Apr 18 15:47:01 MDT 2022
  **** IP Build 3524634 on Mon Apr 18 20:55:01 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source design.tcl
couldn't read file "design.tcl": no such file or directory
INFO: [Common 17-206] Exiting Vivado at Wed Apr 16 07:25:11 2025...
Option Map File Used: '/home/Xilinx/Vitis/2022.1/data/vitis/vpp/optMap.xml'

****** v++ v2022.1 (64-bit)
  **** SW Build 3524075 on 2022-04-13-17:42:45
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

INFO: [v++ 60-1306] Additional information associated with this v++ link can be found at:
	Reports: /home/csl/sky/hls4ml_practice/hls4ml_prj_u55c_new/xclbin_files/_x/reports/link
	Log files: /home/csl/sky/hls4ml_practice/hls4ml_prj_u55c_new/xclbin_files/_x/logs/link
Running Dispatch Server on port: 35001
INFO: [v++ 60-1548] Creating build summary session with primary output /home/csl/sky/hls4ml_practice/hls4ml_prj_u55c_new/xclbin_f